# Imports

In [1]:
import pandas as pd
import os
import math
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import data_prep_utils
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error

# Task 1

### Beat Drug

In [2]:
beat_drug_fp = os.path.join(os.getcwd(), "medical_genomics_2021_data", "beat_drug")
beat_drug_df = pd.read_csv(beat_drug_fp, sep='\t', lineterminator='\n')
print(beat_drug_df.shape)
beat_drug_df.head()

(79, 198)


,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
A-674563,0.302313,0.183573,10.0,2.250759,0.312102,1.112644,3.699257,1.522665,5.132768,1.83987,...,0.726144,0.454933,0.103835,0.192891,0.515219,0.531155,1.261865,0.674077,0.518016,0.259479
Afatinib (BIBW-2992),0.270839,0.095105,10.0,1.042656,0.412103,0.492191,1.126676,0.313330,4.442185,1.09213,...,1.146014,1.047075,0.556972,0.615777,0.456265,0.809377,1.279045,0.983270,2.017039,1.303088
Alisertib (MLN8237),2.292925,10.000000,10.0,10.000000,1.731348,6.149094,10.000000,10.000000,10.000000,10.00000,...,10.000000,10.000000,2.080804,8.928137,6.927002,9.425326,4.785701,10.000000,10.000000,10.000000
Axitinib (AG-013736),0.137656,10.000000,10.0,10.000000,0.124162,10.000000,10.000000,10.000000,10.000000,10.00000,...,10.000000,10.000000,10.000000,10.000000,0.418123,10.000000,10.000000,10.000000,10.000000,0.299026
AZD1480,2.429158,NaN,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,...,10.000000,6.874162,10.000000,10.000000,3.607693,7.004078,10.000000,10.000000,10.000000,10.000000


In [3]:
beat_drug_df.describe()

,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
count,69.000000,70.000000,78.000000,79.000000,79.000000,78.000000,79.000000,79.000000,79.000000,79.000000,...,79.000000,77.000000,78.000000,79.000000,77.000000,78.000000,77.000000,79.000000,79.000000,78.000000
mean,2.412734,6.335849,8.527381,7.409570,3.264417,5.883175,6.617408,6.929836,8.324497,7.984273,...,8.255971,6.404386,6.044692,6.118522,3.080462,7.342661,7.688149,7.247408,8.262772,4.240337
std,3.775972,4.299582,3.146445,3.745823,4.025590,4.275380,3.974038,4.222027,3.351549,3.474718,...,3.330924,4.001485,4.466019,4.325312,3.899469,3.839974,3.755343,3.908462,3.363649,4.264927
min,0.001400,0.006859,0.013717,0.013717,0.005953,0.013717,0.013717,0.013717,0.006859,0.189100,...,0.064853,0.013717,0.012358,0.013717,0.002706,0.097225,0.013717,0.042784,0.111219,0.010583
25%,0.118563,1.085082,10.000000,3.894494,0.248112,1.028161,2.198463,1.468053,10.000000,7.130972,...,9.681701,1.927824,0.918891,1.000000,0.220118,3.142175,5.000000,4.222760,10.000000,0.459808
50%,0.283653,10.000000,10.000000,10.000000,0.897805,6.654102,10.000000,10.000000,10.000000,10.000000,...,10.000000,9.102755,10.000000,10.000000,0.702216,10.000000,10.000000,10.000000,10.000000,1.825458
75%,2.292925,10.000000,10.000000,10.000000,7.096167,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,5.840653,10.000000,10.000000,10.000000,10.000000,10.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [4]:
beat_drug_df.isna().sum().sum()

378

#### Transformation

In [5]:
trans_beat_drug = beat_drug_df.fillna(0.0).apply(np.log10).transpose().reset_index().rename(columns={"index": "sampleID"}) # ToDo: Change fillna
trans_beat_drug = trans_beat_drug.replace(-math.inf, 0.0).replace(math.inf, 1) # ToDo: Deal with that
trans_beat_drug.head()

,sampleID,A-674563,Afatinib (BIBW-2992),Alisertib (MLN8237),Axitinib (AG-013736),AZD1480,Barasertib (AZD1152-HQPA),BEZ235,BMS-345541,Bortezomib (Velcade),...,TG100-115,Tofacitinib (CP-690550),Tozasertib (VX-680),Trametinib (GSK1120212),Vandetanib (ZD6474),Vargetef,Vatalanib (PTK787),Vismodegib (GDC-0449),VX-745,YM-155
0,12-00066,-0.519543,-0.567290,0.360390,-0.861206,0.385456,1.0,-1.417073,0.095660,-0.744268,...,0.502356,0.869371,0.296700,-1.862741,0.216112,-1.333854,0.088790,0.0,-1.106552,-0.661768
1,12-00150,-0.736191,-1.021795,1.000000,1.000000,0.000000,1.0,1.000000,-0.102270,-0.996701,...,1.000000,1.000000,1.000000,1.000000,0.530730,-0.051009,0.409831,0.0,1.000000,-0.197125
2,12-00211,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.888241,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
3,12-00258,0.352329,0.018141,1.000000,1.000000,1.000000,1.0,0.511842,0.515246,0.310652,...,1.000000,-1.862741,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
4,12-00294,-0.505703,-0.384994,0.238384,-0.906013,1.000000,1.0,-1.597193,-0.022079,-0.317022,...,1.000000,0.245287,-0.136855,-1.862741,0.858581,-0.771062,0.588563,1.0,-0.256898,-0.009582


### Beat RNASeq

In [6]:
beat_rnaseq_fp = os.path.join(os.getcwd(), "medical_genomics_2021_data", "beat_rnaseq")
beat_rnaseq_df = pd.read_csv(beat_rnaseq_fp, sep='\t', lineterminator='\n')
print(beat_rnaseq_df.shape)
beat_rnaseq_df.head()

(22843, 198)


,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
TSPAN6,0.070215,1.709459,0.000000,0.000000,0.000000,0.921085,0.000000,0.000000,0.000000,0.115955,...,0.052768,0.271781,0.118997,0.164221,0.648789,0.085224,0.395749,0.249059,0.142059,0.150957
DPM1,23.311415,32.137834,25.117470,29.351673,31.240770,22.940781,28.176101,34.303196,39.203413,24.089688,...,21.089525,25.547442,26.208989,32.023086,34.238344,28.607021,25.101824,38.785308,33.703438,16.575035
SCYL3,10.532266,17.778376,37.659392,15.647852,39.189992,33.734750,23.989937,17.804722,15.450059,25.510139,...,6.842223,34.244444,30.433366,26.234297,13.477109,20.823184,17.950065,7.245358,16.461058,16.212740
C1orf112,11.129094,11.253940,12.541923,8.190423,18.875313,11.110592,12.907338,6.758411,23.516117,12.465182,...,8.864988,24.520713,14.755571,13.219787,5.721136,16.874449,6.105849,4.845333,8.647826,6.944004
FGR,906.792960,49.004499,111.834945,1004.458236,267.852800,170.631056,68.186164,619.473686,11.031520,201.790997,...,451.393235,187.045931,102.961759,186.329197,583.113507,22.300408,24.508199,77.751751,45.565344,51.898882


In [7]:
beat_rnaseq_df.describe()

,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
count,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,...,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000
mean,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,...,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087
std,239.036200,179.037496,199.633444,255.030349,188.159584,165.383966,181.704322,357.619163,204.698390,190.984151,...,287.677166,262.096631,261.028608,229.596117,215.486602,290.598365,498.113439,430.670074,364.906680,272.110655
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.315968,0.398874,0.437118,0.286824,0.457980,0.546894,0.295178,0.255570,0.415165,0.434832,...,0.228660,0.483167,0.475986,0.492663,0.530827,0.426122,0.339214,0.316984,0.248603,0.332105
50%,3.896938,4.558558,5.245415,3.888061,5.659323,5.871919,4.535010,3.577982,4.181302,5.536860,...,2.603211,4.922261,4.581367,5.296126,4.305597,4.460082,2.996389,3.418903,3.480439,4.106020
75%,29.560559,32.565199,37.121401,30.435231,38.601161,37.534227,36.575262,29.986899,29.891861,38.047793,...,24.035727,33.489496,32.248063,34.979063,28.119088,32.939266,20.819249,23.762511,30.862263,28.591181
max,18392.670950,8743.627777,15104.913180,25132.075360,14059.753090,8427.297469,13429.990780,44188.081700,10933.570860,12421.379490,...,17424.573900,19765.174310,16845.031580,15343.553510,13268.700230,20064.259830,42172.561690,34482.244890,42426.733510,13589.053420


In [8]:
beat_rnaseq_df.isna().sum().sum()

0

#### Transformation

In [9]:
trans_beat_rna = beat_rnaseq_df.fillna(0.0).transform(lambda x: x + 1).apply(np.log2).transpose().reset_index().rename(columns={"index": "sampleID"})
trans_beat_rna = trans_beat_rna.replace(-math.inf, 0.0).replace(math.inf, 1) # ToDo: Deal with that
trans_beat_rna.iloc[:, :100].describe()
trans_beat_rna.head()

,sampleID,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,RP11-218F10.3,RP11-686O6.2,RP11-313P22.1,RP11-548H3.1,RP11-102N12.3,RP11-196O16.1,RP4-671G15.2,RP11-731C17.2,RP4-621B10.8,RP11-114I8.4
0,12-00066,0.097901,4.603562,3.527604,3.600400,9.826219,2.520930,5.487015,5.502739,5.868784,...,0.576770,0.189578,0.275776,1.327472,0.275776,0.396124,0.144468,1.200994,0.049781,0.643137
1,12-00150,1.438005,5.050407,4.231000,3.615174,5.643986,3.211384,6.419366,5.334356,5.046681,...,0.192137,0.702038,0.192137,1.208258,1.311231,1.058426,0.000000,0.702038,0.000000,0.227674
2,12-00211,0.000000,4.706943,5.272747,3.759361,6.818070,1.640779,5.908769,6.094865,6.294083,...,1.329468,0.381416,0.182054,1.609326,0.093897,0.981932,0.093897,1.857068,0.454038,1.386245
3,12-00258,0.000000,4.923704,4.057264,3.200131,9.973637,4.972847,4.955170,5.492224,5.719514,...,0.363815,0.870742,0.173099,1.865981,0.654113,0.433561,0.089144,0.967966,0.467209,0.290525
4,12-00294,0.000000,5.010814,5.328764,4.312906,8.070673,1.760450,5.260227,5.843724,5.932174,...,1.566947,0.258552,0.297473,1.689064,0.511232,0.046439,0.297473,2.513655,0.862210,0.913222


#### Sum to 1

In [10]:
trans_beat_rna.sum(axis=0)

sampleID         12-0006612-0015012-0021112-0025812-0029412-003...
TSPAN6                                                   74.008268
DPM1                                                    972.906802
SCYL3                                                   845.939414
C1orf112                                                687.279327
                                       ...                        
RP11-196O16.1                                           138.154865
RP4-671G15.2                                             31.551622
RP11-731C17.2                                           323.120805
RP4-621B10.8                                             36.094048
RP11-114I8.4                                            126.045445
Length: 22844, dtype: object

## Join

In [11]:
beat_rna_and_drug = pd.merge(trans_beat_rna, trans_beat_drug, on="sampleID")
print(f"Num of samples in rna: {trans_beat_rna.shape[0]}\nNum of samples in drug: {trans_beat_drug.shape[0]}\nNum of samples in joined: {beat_rna_and_drug.shape[0]}")
beat_rna_and_drug.head()
print(np.isinf(beat_rna_and_drug.iloc[:, 1:]).values.sum())
print(f"Cols in rna: {len(trans_beat_rna.columns)}\nCols in drugs: {len(trans_beat_drug.columns)}\
    \nCols in merged: {len(beat_rna_and_drug.columns)}")
print(trans_beat_rna.columns[-1])
print(trans_beat_drug.columns[1])
print(beat_rna_and_drug.columns[22843:22846])
drugs_reorg = beat_rna_and_drug.iloc[:, list(range(22844, 22923)) + [0]]
print(len(drugs_reorg.columns))
beat_rna_and_drug.head()

Num of samples in rna: 198
Num of samples in drug: 198
Num of samples in joined: 198
0
Cols in rna: 22844
Cols in drugs: 80    
Cols in merged: 22923
RP11-114I8.4
A-674563
Index(['RP11-114I8.4', 'A-674563', 'Afatinib (BIBW-2992)'], dtype='object')
80


,sampleID,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,TG100-115,Tofacitinib (CP-690550),Tozasertib (VX-680),Trametinib (GSK1120212),Vandetanib (ZD6474),Vargetef,Vatalanib (PTK787),Vismodegib (GDC-0449),VX-745,YM-155
0,12-00066,0.097901,4.603562,3.527604,3.600400,9.826219,2.520930,5.487015,5.502739,5.868784,...,0.502356,0.869371,0.296700,-1.862741,0.216112,-1.333854,0.088790,0.0,-1.106552,-0.661768
1,12-00150,1.438005,5.050407,4.231000,3.615174,5.643986,3.211384,6.419366,5.334356,5.046681,...,1.000000,1.000000,1.000000,1.000000,0.530730,-0.051009,0.409831,0.0,1.000000,-0.197125
2,12-00211,0.000000,4.706943,5.272747,3.759361,6.818070,1.640779,5.908769,6.094865,6.294083,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
3,12-00258,0.000000,4.923704,4.057264,3.200131,9.973637,4.972847,4.955170,5.492224,5.719514,...,1.000000,-1.862741,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
4,12-00294,0.000000,5.010814,5.328764,4.312906,8.070673,1.760450,5.260227,5.843724,5.932174,...,1.000000,0.245287,-0.136855,-1.862741,0.858581,-0.771062,0.588563,1.0,-0.256898,-0.009582


## Linear Regression

In [12]:
t_1_LR = LinearRegression()
num_of_features = beat_rnaseq_df.shape[0]
print(f"Num of features (genes): {num_of_features}")
X = beat_rna_and_drug.iloc[:, 1:num_of_features + 1]
y = beat_rna_and_drug.iloc[:, num_of_features + 2]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
t_1_LR.fit(x_train, y_train)
y_pred = t_1_LR.predict(x_test)
score = r2_score(y_test, y_pred)
print(score)

Num of features (genes): 22843
-0.22014216665460618


# Beat X TCGA

In [13]:
beat_rnaseq_df = data_prep_utils.get_df("beat_rnaseq")
tcga_rna = data_prep_utils.get_df("tcga_rna")

In [14]:
beat_rnaseq_df = beat_rnaseq_df.reset_index().rename(columns={"index": "gene"})
tcga_rna = tcga_rna.reset_index().rename(columns={"index": "gene"})

In [15]:
tcga_rna.head()

,gene,TCGA.AB.2803,TCGA.AB.2805,TCGA.AB.2806,TCGA.AB.2807,TCGA.AB.2808,TCGA.AB.2810,TCGA.AB.2811,TCGA.AB.2812,TCGA.AB.2813,...,TCGA.AB.3000,TCGA.AB.3001,TCGA.AB.3002,TCGA.AB.3005,TCGA.AB.3006,TCGA.AB.3007,TCGA.AB.3008,TCGA.AB.3009,TCGA.AB.3011,TCGA.AB.3012
0,A1BG,21.786275,5.251160,10.866631,18.869409,8.199836,13.152894,7.799243,7.607058,13.230136,...,12.968585,16.882928,13.293426,8.819212,5.905764,18.960694,8.283968,7.088477,12.933981,15.191255
1,A1CF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055844,0.000000,0.000000
2,A2BP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,A2LD1,3.772166,3.007237,2.951634,2.491801,2.934105,1.334901,1.770624,5.297683,3.233179,...,1.918685,1.936251,1.171619,5.119481,2.640735,3.583209,2.029229,6.645094,1.965216,1.593010
4,A2M,24.673141,0.995091,0.406560,18.714176,15.931201,0.138116,0.198614,0.108805,12.367011,...,2.423215,9.456122,3.107597,4.403161,5.800682,0.544561,46.349288,14.462855,1.439471,27.238767


In [16]:
beat_rnaseq_df.head()

,gene,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
0,TSPAN6,0.070215,1.709459,0.000000,0.000000,0.000000,0.921085,0.000000,0.000000,0.000000,...,0.052768,0.271781,0.118997,0.164221,0.648789,0.085224,0.395749,0.249059,0.142059,0.150957
1,DPM1,23.311415,32.137834,25.117470,29.351673,31.240770,22.940781,28.176101,34.303196,39.203413,...,21.089525,25.547442,26.208989,32.023086,34.238344,28.607021,25.101824,38.785308,33.703438,16.575035
2,SCYL3,10.532266,17.778376,37.659392,15.647852,39.189992,33.734750,23.989937,17.804722,15.450059,...,6.842223,34.244444,30.433366,26.234297,13.477109,20.823184,17.950065,7.245358,16.461058,16.212740
3,C1orf112,11.129094,11.253940,12.541923,8.190423,18.875313,11.110592,12.907338,6.758411,23.516117,...,8.864988,24.520713,14.755571,13.219787,5.721136,16.874449,6.105849,4.845333,8.647826,6.944004
4,FGR,906.792960,49.004499,111.834945,1004.458236,267.852800,170.631056,68.186164,619.473686,11.031520,...,451.393235,187.045931,102.961759,186.329197,583.113507,22.300408,24.508199,77.751751,45.565344,51.898882


In [17]:
beat_rnaseq_x_tcga = beat_rnaseq_df[beat_rnaseq_df["gene"].isin(tcga_rna["gene"])]

In [18]:
print(f"{beat_rnaseq_df.shape}\n{beat_rnaseq_x_tcga.shape}")

(22843, 199)
(14187, 199)


In [19]:
beat_rnaseq_x_tcga.set_index("gene", inplace=True)
# beat_rnaseq_x_tcga.columns = beat_rnaseq_x_tcga.columns.to_flat_index()

In [20]:
beat_rnaseq_x_tcga.head()

,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0.070215,1.709459,0.000000,0.000000,0.000000,0.921085,0.000000,0.000000,0.000000,0.115955,...,0.052768,0.271781,0.118997,0.164221,0.648789,0.085224,0.395749,0.249059,0.142059,0.150957
DPM1,23.311415,32.137834,25.117470,29.351673,31.240770,22.940781,28.176101,34.303196,39.203413,24.089688,...,21.089525,25.547442,26.208989,32.023086,34.238344,28.607021,25.101824,38.785308,33.703438,16.575035
SCYL3,10.532266,17.778376,37.659392,15.647852,39.189992,33.734750,23.989937,17.804722,15.450059,25.510139,...,6.842223,34.244444,30.433366,26.234297,13.477109,20.823184,17.950065,7.245358,16.461058,16.212740
C1orf112,11.129094,11.253940,12.541923,8.190423,18.875313,11.110592,12.907338,6.758411,23.516117,12.465182,...,8.864988,24.520713,14.755571,13.219787,5.721136,16.874449,6.105849,4.845333,8.647826,6.944004
FGR,906.792960,49.004499,111.834945,1004.458236,267.852800,170.631056,68.186164,619.473686,11.031520,201.790997,...,451.393235,187.045931,102.961759,186.329197,583.113507,22.300408,24.508199,77.751751,45.565344,51.898882


In [21]:
beat_rnaseq_x_tcga_trans = data_prep_utils.transpose_df(beat_rnaseq_x_tcga)

In [22]:
beat_rnaseq_x_tcga_trans.head()

gene,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,NIPAL3,...,TERC,TSIX,SRXN1,SNORD11B,GTF2H5,NUDT3,SNORA28,DOC2B,SNURF,DGCR11
12-00066,0.070215,23.311415,10.532266,11.129094,906.792960,4.739520,43.849332,44.340838,57.435955,25.207222,...,0.105323,0.000000,1.053227,0.035108,10.181190,246.770983,0.280860,8.109845,0.070215,2.457529
12-00150,1.709459,32.137834,17.778376,11.253940,49.004499,8.262386,84.589743,39.346054,32.052361,23.020718,...,0.256419,0.000000,0.769257,0.028491,7.721058,161.116537,0.113964,0.455856,0.284910,1.851914
12-00211,0.000000,25.117470,37.659392,12.541923,111.834945,2.118341,59.078171,67.349788,77.470749,71.250225,...,0.235371,1.513101,0.638865,0.033624,11.196944,274.644567,0.605240,3.900437,0.235371,2.622708
12-00258,0.000000,29.351673,15.647852,8.190423,1004.458236,30.403362,30.020930,44.011575,51.692089,28.299985,...,0.223085,0.223085,1.402251,0.000000,11.887268,106.475505,0.223085,0.350563,0.159347,1.784684
12-00294,0.000000,31.240770,39.189992,18.875313,267.852800,2.388038,37.325360,56.429663,60.060789,43.671652,...,0.163564,0.000000,0.032713,0.130851,11.842051,272.759727,0.130851,0.000000,0.327128,1.995484


In [23]:
beat_rnaseq_x_tcga_trans.columns[0]

'TSPAN6'

# Sum to 1

In [24]:
beat_rnaseq_df = data_prep_utils.get_df("beat_rnaseq")
beat_rnaseq_trans = data_prep_utils.transpose_df(beat_rnaseq_df)

In [25]:
beat_rnaseq_trans.head(10)

,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,RP11-218F10.3,RP11-686O6.2,RP11-313P22.1,RP11-548H3.1,RP11-102N12.3,RP11-196O16.1,RP4-671G15.2,RP11-731C17.2,RP4-621B10.8,RP11-114I8.4
12-00066,0.070215,23.311415,10.532266,11.129094,906.792960,4.739520,43.849332,44.340838,57.435955,4.177799,...,0.491506,0.140430,0.210645,1.509625,0.210645,0.315968,0.105323,1.298979,0.035108,0.561721
12-00150,1.709459,32.137834,17.778376,11.253940,49.004499,8.262386,84.589743,39.346054,32.052361,10.199774,...,0.142455,0.626802,0.142455,1.310585,1.481531,1.082658,0.000000,0.626802,0.000000,0.170946
12-00211,0.000000,25.117470,37.659392,12.541923,111.834945,2.118341,59.078171,67.349788,77.470749,7.767250,...,1.513101,0.302620,0.134498,2.051092,0.067249,0.975109,0.067249,2.622708,0.369869,1.613974
12-00258,0.000000,29.351673,15.647852,8.190423,1004.458236,30.403362,30.020930,44.011575,51.692089,5.035358,...,0.286824,0.828603,0.127477,2.645156,0.573648,0.350563,0.063739,0.956081,0.382432,0.223085
12-00294,0.000000,31.240770,39.189992,18.875313,267.852800,2.388038,37.325360,56.429663,60.060789,6.379005,...,1.962771,0.196277,0.228990,2.224474,0.425267,0.032713,0.228990,4.710650,0.817821,0.883247
12-00372,0.921085,22.940781,33.734750,11.110592,170.631056,148.841632,36.267735,65.742465,79.587529,10.966672,...,0.834734,0.863517,0.777166,3.454070,0.287839,1.151357,0.172703,3.943397,0.287839,0.863517
13-00028,0.000000,28.176101,23.989937,12.907338,68.186164,0.858700,47.872537,85.950524,80.234801,8.801677,...,0.617191,1.556394,0.187841,1.368553,0.241509,0.241509,0.134172,1.046541,0.268344,0.134172
13-00034,0.000000,34.303196,17.804722,6.758411,619.473686,1.419834,86.524700,33.536485,31.491924,11.216691,...,0.425950,1.277851,0.113587,1.107471,0.908694,0.028397,0.085190,1.079074,0.113587,0.056793
13-00098,0.000000,39.203413,15.450059,23.516117,11.031520,1.719968,42.168875,58.953392,80.601268,3.350972,...,0.118618,0.266892,0.118618,3.321318,0.088964,0.415165,0.000000,2.253751,0.029655,0.355855
13-00118,0.115955,24.089688,25.510139,12.465182,201.790997,7.334165,53.223426,62.441863,89.807285,9.827201,...,1.101574,0.057978,0.144944,2.840902,0.057978,0.637753,0.028989,2.898879,0.086966,1.188541


In [26]:
beat_rnaseq_trans.iloc[:, 1:] = beat_rnaseq_trans.iloc[:, 1:].divide(beat_rnaseq_trans.iloc[:, 1:].sum(axis=0).to_numpy(), axis=1)


In [27]:
print(beat_rnaseq_trans.DPM1.sum())
beat_rnaseq_trans.head()

1.0000000000000002


,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,RP11-218F10.3,RP11-686O6.2,RP11-313P22.1,RP11-548H3.1,RP11-102N12.3,RP11-196O16.1,RP4-671G15.2,RP11-731C17.2,RP4-621B10.8,RP11-114I8.4
12-00066,0.070215,0.003928,0.002632,0.004891,0.013171,0.001509,0.004782,0.004736,0.004720,0.003753,...,0.003404,0.001028,0.006223,0.004159,0.002921,0.002092,0.004358,0.002713,0.001219,0.004699
12-00150,1.709459,0.005416,0.004442,0.004945,0.000712,0.002631,0.009225,0.004203,0.002634,0.009163,...,0.000987,0.004590,0.004208,0.003611,0.020541,0.007167,0.000000,0.001309,0.000000,0.001430
12-00211,0.000000,0.004233,0.009410,0.005511,0.001624,0.000674,0.006443,0.007194,0.006366,0.006978,...,0.010479,0.002216,0.003973,0.005651,0.000932,0.006455,0.002783,0.005478,0.012841,0.013501
12-00258,0.000000,0.004946,0.003910,0.003599,0.014590,0.009680,0.003274,0.004701,0.004248,0.004524,...,0.001986,0.006068,0.003766,0.007287,0.007953,0.002321,0.002637,0.001997,0.013277,0.001866
12-00294,0.000000,0.005265,0.009793,0.008295,0.003891,0.000760,0.004071,0.006027,0.004936,0.005731,...,0.013594,0.001437,0.006765,0.006128,0.005896,0.000217,0.009475,0.009839,0.028393,0.007389


In [28]:
test_dict = {"sampleID": ["a", "b", "c", "d"], "gene1": [1, 2, 3, 4], "gene2": [5, 6, 7, 8]}
test_df = pd.DataFrame.from_dict(test_dict)
test_df

,sampleID,gene1,gene2
0,a,1,5
1,b,2,6
2,c,3,7
3,d,4,8


In [29]:
print(test_df.iloc[:, 1:].sum(axis=0).to_numpy())
test_df.iloc[:, 1:] = test_df.iloc[:, 1:].divide(test_df.iloc[:, 1:].sum(axis=0).to_numpy(), axis=1)
test_df

[10 26]


,sampleID,gene1,gene2
0,a,0.1,0.192308
1,b,0.2,0.230769
2,c,0.3,0.269231
3,d,0.4,0.307692


In [30]:
test_df.iloc[:, 1:] = test_df.iloc[:, 1:].transform(lambda x: x * 1000000)
test_df.sum(axis=0)

sampleID         abcd
gene1       1000000.0
gene2       1000000.0
dtype: object

# Missing Values

In [31]:
missing_test_dict = {"s1": [0, 1, 2, 3, None], "s2": [4, 5, None, 6, 7], "s3": [8, None, 9, None, 10]}
missing_test_df = pd.DataFrame.from_dict(missing_test_dict)
missing_test_df.transpose()

,0,1,2,3,4
s1,0.0,1.0,2.0,3.0,NaN
s2,4.0,5.0,NaN,6.0,7.0
s3,8.0,NaN,9.0,NaN,10.0


In [32]:
missing_test_df_t = missing_test_df.transpose()
values = missing_test_df_t.max
missing_test_df_t.fillna(value=values().to_dict())

,0,1,2,3,4
s1,0.0,1.0,2.0,3.0,10.0
s2,4.0,5.0,9.0,6.0,7.0
s3,8.0,5.0,9.0,6.0,10.0


# Missing kmeans

In [33]:
miss_k_dict = {"index": ["a", "b", "c", "d", "e"], "s1": [0, 1, 2, 3, None], "s2": [0, 1, None, 3, 4], "s3": [8, None, 9, None, 10], "s4": [8, 9, None, 11, 10]}
miss_k_df = pd.DataFrame.from_dict(miss_k_dict).set_index("index")
miss_k_df

,s1,s2,s3,s4
index,,,,
a,0.0,0.0,8.0,8.0
b,1.0,1.0,NaN,9.0
c,2.0,NaN,9.0,NaN
d,3.0,3.0,NaN,11.0
e,NaN,4.0,10.0,10.0


In [34]:
imputer = KNNImputer(n_neighbors=1)
miss_k_df.iloc[:, :] = imputer.fit_transform(miss_k_df)
miss_k_df

,s1,s2,s3,s4
index,,,,
a,0.0,0.0,8.0,8.0
b,1.0,1.0,8.0,9.0
c,2.0,1.0,9.0,9.0
d,3.0,3.0,9.0,11.0
e,2.0,4.0,10.0,10.0


# Filter by tcga

In [35]:
filter_dict1 = {"g1": [0, 1, 2, 3, 4], "g2": [4, 5, 6, 6, 7], "g3": [8, 8, 9, 9, 10]}
filter_dict2 = {"g1": [0, 1, 2, 3, 4], "g2": [4, 5, 6, 6, 7]}

filter_df1 = pd.DataFrame.from_dict(filter_dict1, orient="index", columns=["s1", "s2", "s3", "s4", "s5"])
filter_df2 = pd.DataFrame.from_dict(filter_dict2, orient="index", columns=["s1", "s2", "s3", "s4", "s5"])

print(filter_df1)
print(filter_df2)

    s1  s2  s3  s4  s5
g1   0   1   2   3   4
g2   4   5   6   6   7
g3   8   8   9   9  10
    s1  s2  s3  s4  s5
g1   0   1   2   3   4
g2   4   5   6   6   7


In [36]:
filter_df1[filter_df1.index.isin(filter_df2.index)]

,s1,s2,s3,s4,s5
g1,0,1,2,3,4
g2,4,5,6,6,7


# Divide to folds

In [37]:
division_fp = os.path.join(os.getcwd(), "folds.txt")
division_df = pd.read_csv(division_fp, sep='\t', lineterminator='\n', header=None, names=["sampleID", "fold"])
division_df.head()
genes_df = data_prep_utils.get_df("beat_rnaseq").transpose()
# division_df[division_df["fold"]==1]["sampleID"]
genes_df[genes_df.index.isin(division_df[division_df["fold"]==5]["sampleID"])].shape

(39, 22843)

# Calc MSE

In [39]:
true_y_dict = {"d1": [0, 1, 2, 3, 4], "d2": [4, 5, 6, 6, 7], "d3": [8, 8, 9, 9, 10]}
true_y_df = pd.DataFrame.from_dict(true_y_dict)#, orient="index", columns=["s1", "s2", "s3", "s4", "s5"])
print(true_y_df)

pred_y_dict = {"d1": [0, 1, 2, 3, 4], "d2": [4, 5, 6, 6, 7], "d3": [9, 7, 9.7, 8.2, 9]}
pred_y_df = pd.DataFrame.from_dict(pred_y_dict)#, orient="index", columns=["s1", "s2", "s3", "s4", "s5"])
print(pred_y_df)

mean_squared_error(true_y_df.transpose(), pred_y_df.transpose())


   d1  d2  d3
0   0   4   8
1   1   5   8
2   2   6   9
3   3   6   9
4   4   7  10
   d1  d2   d3
0   0   4  9.0
1   1   5  7.0
2   2   6  9.7
3   3   6  8.2
4   4   7  9.0


0.2753333333333333